In [18]:
import math, pickle, os, copy, sys, scipy.io
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import random
import matplotlib.pyplot as plt
from sklearn import linear_model

import xgboost as xgb
from sklearn.metrics import r2_score, mean_squared_error

## Get the data

In [19]:
#### Directory where the env is ####
filename_env = "../PV V2/Synthetic_PV_profiles/saved_results/PV_UniModal_nicolas_env_raw_150"

file = open(filename_env, 'rb') 
env_dict = pickle.load(file) 
msg = '[INFO] loaded data for {:2.0f} clients'.format(env_dict['num_clients'])

print(msg)

[INFO] loaded data for 24 clients


In [20]:
clients_data = env_dict['train_scenarios']['sml']['clients_data']

## FedXGBoost 

### Training

In [23]:
%%time

####### How many trees #######
n_trees = 50


clients = []
tests = []

### Create the distributed data ###
for i in range(len(clients_data)):
    
    client_i = clients_data[i]
    

    mat_train = np.matrix(client_i[0])
    client = pd.DataFrame(mat_train)

    y = [client_i[1][i].item() for i in range(len(client_i[1]))]

    client['y'] = y
    
    clients.append(client)

    mat_test = np.matrix(client_i[2])
    test = pd.DataFrame(mat_test)

    y = [client_i[3][i].item() for i in range(len(client_i[3]))]

    test['y'] = y
    tests.append(test)


################## Tree 0 ###################
f_0 = 0
f_m = [f_0]*len(clients)

trees = []   #List of all the trees of the model

feature = list(clients[1].columns)
del feature[len(feature) - 1:]


################## Other trees ###################
for t in range(0,n_trees):
    
    hs = [] #List of all client histogram representation
    
    for i in range(len(clients)):
        
        h = [] # List of all client features histogram representation
    
        clients[i]["g"] = 2*(clients[i]["y"]-f_m[i])*clients[i]["y"]
        clients[i]["h"] = -2*clients[i]["y"]

        for j in range(0,9):
            h.append(clients[i][[j,"g","h"]])
        hs.append(h)
        
       
    print("got the clients")
    
    agged = [] #List of feature histogram at server level 
    
    #Histogram merging
    for j in range(len(feature)):
        feature_dat = hs[0][j]
        for i in range(1,len(clients)):
            feature_dat = feature_dat.append(hs[i][j])
            
        agg = merge_hist(feature_dat, 50,feature[j])
        agged.append(agg)

        
    print("got the server")    
    
    #Get the tree structure
    tree_hist = split(agged,feature,0,2)
    
   
    # Get the leaf weights
    tr = find_leaf_weights_cl(clients, tree_hist)

    #Add the new tree to the list of trees
    trees.append(tr)
    
    print("got the tree")
        
    #Update the predictions for each clients
    for i in range(len(clients)):
        preds = clients[i].apply(predict, axis='columns', rules=tr.copy())
        f_m[i] = f_m[i]+preds    
        
    print(t)

got the clients
got the server
got the tree
0
got the clients
got the server
got the tree
1
got the clients
got the server
got the tree
2
got the clients
got the server
got the tree
3
got the clients
got the server
got the tree
4
got the clients
got the server
got the tree
5
got the clients
got the server
got the tree
6
got the clients
got the server
got the tree
7
got the clients
got the server
got the tree
8
got the clients
got the server
got the tree
9
got the clients
got the server
got the tree
10
got the clients
got the server
got the tree
11
got the clients
got the server
got the tree
12
got the clients
got the server
got the tree
13
got the clients
got the server
got the tree
14
got the clients
got the server
got the tree
15
got the clients
got the server
got the tree
16
got the clients
got the server
got the tree
17
got the clients
got the server
got the tree
18
got the clients
got the server
got the tree
19
got the clients
got the server
got the tree
20
got the clients
got the

### Testing 

In [24]:
################## Predicting using model ###################
res = []
for j in range(len(tests)):  #For each client
    
    preds = 0 #Tree 0
    for i in range(len(trees)): # For each tree
        preds += tests[j].apply(predict, axis='columns', rules=trees[i].copy())

    ################## Calculate RMSE ###################
    res.append(np.sqrt(np.sum(np.power(preds-tests[j]["y"],2))/len(preds)))

    
# Get mean and std of results
mean = np.mean(res)
std = np.std(res)/mean*100

print("RMSE : " + mean.astype(str) + " +- " + std.astype(str) + "%")

RMSE : 27.33677354351649 +- 8.932330875949022%


## Functions

In [5]:
def find_best_rule(hist,features):
    
    #X_train with X, g and h for all samples
    
    best_feature, best_threshold, opt_split = None, None, np.inf

    for i in range(len(hist)):
        feat = features[i]
        df = hist[i]
        thresholds = df[feat].unique().tolist()
        thresholds.sort()
        thresholds = thresholds[1:]
        
        #print(len(thresholds))
        t_hist = []
        split_hist = []
        for t in thresholds:
            df_L =  df[df[feat]<t]
            df_R =  df[df[feat]>=t]
    
            G = np.sum(df["g"])
            H = np.sum(df["h"])
    
            G_L = np.sum(df_L["g"])
            G_R = np.sum(df_R["g"])
    
            H_L = np.sum(df_L["h"])
            H_R = np.sum(df_R["h"])
            
              
            if (H==0) or (H_R==0) or (H_L ==0):

                continue
        
            split = 1/2*((G_L*G_L)/H_L + (G_R*G_R)/H_R - (G*G)/H)
            
            opt_split = min(opt_split, split)
            
            if split == opt_split:
                x_opt = t
                best_feature = feat
                
            t_hist.append(t)
            split_hist.append(split)
    
    
        '''
        plt.figure()
        plt.plot(t_hist,split_hist)
        plt.title(feat)
    
        '''
    return {'feature': best_feature, 'threshold': x_opt}


def split(hist,feature, depth, max_depth):
    '''size = []
    for i in hist:
        size.append(len(size))
    mini = min(size)
    
    print(mini)
    '''
    if depth == max_depth:
        return {'prediction': random.random()}
    
    #Find the best split
    rule = find_best_rule(hist, feature)
    feat = feature.index(rule["feature"])
    
    #Split on the rule
    left_hist = hist.copy()
    right_hist = hist.copy()
    
    left_ix = hist[feat][rule['feature']] < rule['threshold']
    
    left_hist[feat] = hist[feat][left_ix]
    right_hist[feat] = hist[feat][~left_ix]
    
    #Find best rule and split on subsamples
    rule['left'] = split(left_hist, feature, depth + 1, max_depth)
    rule['right'] = split(right_hist, feature, depth + 1, max_depth)
    return rule

def find_leaf_weights_cl(clients,tree):
    
    weights = [None]*len(clients)
    
    
    for i in range(len(clients)):
        clients[i]["tag"] = clients[i].apply(predict, axis='columns', rules=tree.copy())
        weights[i] = clients[i].groupby('tag')[['g','h']].sum()
        
        #weights[i]["w"] = -weights[i]["g"]/weights[i]["h"]
        clients[i].drop('tag', axis = 1, inplace = True)
        #weights[i].drop(["g","h"],axis = 1, inplace = True)

    
    weight = weights[0]
    for i in range(1,len(weights)):
        weight = weight.append(weights[i])
    
    weight = weight.groupby("tag")[["g","h"]].sum()
    
    weight["w"] = -weight["g"]/weight["h"]
    
    weight.drop(["g","h"],axis = 1, inplace = True)
    
    
    paths = [["left","left"],["left","right"],["right","left"],["right","right"]]
    
    for i in paths: 
        target = tree[i[0]][i[1]]["prediction"]
        
        if weight[weight.index == target].empty : 
            tree[i[0]][i[1]]["prediction"] = 0
        else :
            tree[i[0]][i[1]]["prediction"] = weight[weight.index == target].values[0][0]

    return tree


def predict(sample, rules):
    prediction = None
    while prediction is None:
        feature, threshold = rules['feature'], rules['threshold']
        if sample[feature] < threshold:
            rules = rules['left']
        else:
            rules = rules['right']
        prediction = rules.get('prediction', None)
    return prediction


In [6]:
def merge_hist(df,k,feat):
    m = len(df)
    if k>m:
        print("already satisfied")
        return df
    else:
        while k<m:
            df = df.sort_values(by=[feat])
            df.reset_index(inplace = True, drop = True)
            a = list(df[feat])
            b = [j-i for i, j in zip(a[:-1], a[1:])]

            mini = b.index(min(b))

            new = [(a[mini]+a[mini+1])/2,df["g"][mini]+df["g"][mini+1],df["h"][mini]+df["h"][mini+1]]

            df.loc[len(df)] = new
            df.drop([mini,mini+1], inplace = True)
            m = len(df)
        return df
